<a href="https://colab.research.google.com/github/KimHee-Sun/class2022Spring/blob/main/audio_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio Processing
---

https://musiclab.chromeexperiments.com/Spectrogram/

-> 시간축 (x축)의 오르락내리락 하는 파형이 있다고 할 때, 어떤 성분이 많은 지가 중요함 (높은 음, 낮은 음)
=> 이를 설명해주는 게 spectrum
=> 입체적으로 어떤 주파수가 더 많은 지 보여주는 것이 spectrogram
- x축은 시간, **y축은 주파수**, 높은 음쪽이 많으면 노란색/파란색이 나올 것
- 어디에 어떤 성분이 많은 지에 따라 소리가 구분됨 (아에이오우를 주파수로 구분)
- 해당 주파수 대의 sin곡선이 만들어지는 것인데, 우리 주변의 소리 (목소리, 악기소리)는 하나의 주파수로 만들어지지 않음 => 여러개의 성분이 합쳐져있음 (같이 공명하는 다른 소리가 있음)
- wave 상에서는 볼 수 없어서, 반드시 주파수 분석을 해야만 어떤 성분이 많고 적은 지 알 수 있는 것 (e.g. 드럼은 아래에, 새소리는 위에 많이 분포)

  ☝ **y축이 주파수, 주파수 분석이라는 것을 알아야 하고 spectrum 여러장이 시간축으로 계속 나오고 있는 것이 spectrogram인 것임**

### Fourier transform (분석용) -> 어떤 주파수가 많은 지 '분석'하기 위함
* **spectrum은 주어진 signal에 대해 어떤 주파수 성분이 많이 있나?** 
      - given signal에 대해서 어떤 주파수 성분이 많은 지 알아보는 것 
      -> e.g. 오케스트라 특정 시간의 sin곡선이 frqeuncy 우측에서 많으면 높은 음이 많았다는 것을 알 수 있음 (<-> 좌측에 많으면 낮은 음 연주 중)
* **spectrogram은 spectrum을 time 축으로 concatenate한 것**
      - spectrum을 그림으로 그리면 x축이 frequency, y축에 apmplitude -> 이것이 시간 축으로 쭉 입체적으로 내려와 연결이 되면, 위에서 볼 때 아까 같은 그림
      -> x축이 frequency, y축이 amplitude가 되는 한장을 spectrum이라고 하고, (시간이 흘러가니까) 여러개를 쌓아서 시간에 따라 concat하면 spectrogram이 됨 (=concat of spectrum over time)
* **방법: signal (inner product) a series of complex phasors with different frequencies**
      - given signal이 있을 때 여러가지 다양한 sin 곡선을 (complex phasors), 다양한 frequency를 갖도록 만드는 것
* inner product는 일종의 correlation (즉, 해당 frequency가 얼마나 있는지 probing)
* 왜? sine phasor 안 쓰나? phase sensitivity 때문

### Filter (변환용)
* A --> function -->  B
* signal A --> filter --> signal B
* 신호 (time function)를 입력으로 하는 함수를 filter라고 함
* 왜? filter 라고 부름? 이 함수의 목적이 특정 주파수에 대한 manipulation이므로.
(예: 어떤 주파수대를 작게, 크게, 통과, 제거 등)
* 방법: weighted sum of signal's shifts (두가지 방법: FIR, IIR)

  * FIR: Y(k) = b<sub>1</sub>X(k) + b<sub>2</sub>X(k-1) + ... 
    - Y = H * X
  * IIR: a<sub>1</sub>Y(k) + a<sub>2</sub>Y(k-1) + ... = X(k)
    - Y = (1/H) * X

### Auto correlation
* measuring pitch / F0

### RMS: root mean square
* measuring intensity

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import IPython.display as ipd
import librosa, librosa.display

### load / plot / play sound file

In [ ]:
# from google.colab import files
# fn = files.upload()
import os
url = "https://raw.githubusercontent.com/hsnam95/class2022Spring/main/aeiou.wav"
os.system("curl " + url + " > aeiou.wav")

s, sr = librosa.load('aeiou.wav')

In [ ]:
s = librosa.util.normalize(s)
librosa.display.waveplot(s, sr)
ipd.Audio(s[7000:12000], rate=sr)

### Fourier Transform for Spectrogram

In [ ]:
s_preemp = librosa.effects.preemphasis(s)

n_fft=512
hop_length=int(0.001*sr)
win_length=int(sr*0.008)

spec = librosa.stft(s_preemp, n_fft=n_fft, hop_length=hop_length, win_length=win_length, window = 'hann')
magspec = np.abs(spec)
dBspec = librosa.amplitude_to_db(magspec, ref=np.max)

plt.figure(figsize=(15, 5))
librosa.display.specshow(dBspec, sr=sr, x_coords = np.linspace(1, len(s), dBspec.shape[1])/sr , x_axis='time', y_axis='linear', cmap='Greys')
plt.ylim((0,5000))

### Filter for audio transformation

In [ ]:
from scipy.signal import lfilter
sig = s[7000:12000]
sig = lfilter(np.array([1]), np.array([1]), sig, axis=0)
librosa.display.waveplot(sig, sr)
ipd.Audio(sig, rate=sr)

### RMS(Root Mean Square) for intensity

In [ ]:
rms = librosa.feature.rms(s)
plt.plot(rms[0])

### Autocorrelation for pitch(F0) measurement

In [ ]:
F0, voiced_flag, voiced_prob = librosa.pyin(s, 60, 200)
plt.plot(F0, '.')